In [3]:
#create 1d cnn model
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN1d(nn.Module):
    #create a 1d cnn regression model
    #the ouput is a target value of 60 minutes later, is a scalar
    def __init__(self, input_len, input_dim, kernel_size, layer_num, hidden_size,
                  dropout=0.5, batch_norm=True):
        super(CNN1d, self).__init__()
        self.input_len = input_len
        self.input_dim = input_dim
        self.kernel_size = kernel_size
        self.layer_num = layer_num
        self.hidden_size = hidden_size
        self.dropout = dropout
        self.batch_norm = batch_norm
        self.convs = nn.ModuleList([nn.Conv1d(self.input_dim, self.hidden_size, self.kernel_size)])
        self.bns = nn.ModuleList([nn.BatchNorm1d(self.hidden_size)])
        for i in range(self.layer_num - 1):
            self.convs.append(nn.Conv1d(self.hidden_size, self.hidden_size, self.kernel_size))
            self.bns.append(nn.BatchNorm1d(self.hidden_size))
        self.fc1 = nn.Linear(self.hidden_size, 1)
        self.dropout = nn.Dropout(self.dropout)
        self.init_weights()

    def init_weights(self):
        #initialize weights
        for conv in self.convs:
            torch.nn.init.xavier_uniform_(conv.weight)
        torch.nn.init.xavier_uniform_(self.fc1.weight)

    def forward(self, x):
        #forward pass
        for i, conv in enumerate(self.convs):
            if self.batch_norm:
                x = F.relu(self.bns[i](conv(x)))
            else:
                x = F.relu(conv(x))
            x = self.dropout(x)
        x = self.fc1(x[:, :, -1])
        return x


In [5]:
# test case
model = CNN1d(input_len=60, input_dim=10, kernel_size=3, layer_num=2, hidden_size=64)
print(model)
x = torch.randn(32, 10, 60)
y = model(x)
print(y.shape)


CNN1d(
  (convs): ModuleList(
    (0): Conv1d(10, 64, kernel_size=(3,), stride=(1,))
    (1): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  )
  (bns): ModuleList(
    (0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc1): Linear(in_features=64, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
torch.Size([32, 1])
